In [2]:
#Import libraries
import cv2
import smtplib 
#import pywhatkit
import boto3
import time
import numpy as np
from os import listdir
from os.path import isfile, join
import os

## Step1:  create Training data

In [3]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
 #   faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces_user_detec/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 20: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

Face not found
Face not found
Face not found
Face not found
Face not found


## Step2: Train the model

In [4]:
# Get the training data we previously made
data_path = './faces_user_detec/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python

model  = cv2.face_LBPHFaceRecognizer.create()
#model = cv2.
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessfully !!!")

Model trained sucessfully !!!


## Step 3 - Run Our Facial Recognition

In [6]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Face Detected", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
######## Send me mail when face detected
### For that you have to import smtplib library and install it before with "pip install secure-smtplib"
            
            sender = "XXX-XXX@gmail.com"             #Put ur mail id
            receivers = ['XXX-XXX@gmail.com']       #Put the receiver mail id here 
                        #pas = getpass.getpass()
            passwd =  open('../task6/pwd.txt', 'r').read()   #read from my local-file the passwd to access my gmail

            message = """From: From SENDER_NAME <XXX-XXX@gmail.com>
                                    To: RECEIVER_NAME <XXX-XXX@gmail.com>
                                    Subject: Face Unlock Detected

                                    This is a test message To unlock Your device.....
                        """
            try:
#smtpObj = smtplib.SMTP('localhost')
                smtpObj = smtplib.SMTP('smtp.gmail.com', 587)  # Establish a connection to the gmail smtp server
                smtpObj.starttls()    #Establish a secure connection
                smtpObj.login(sender, passwd)   #login to my gmail acc to send an email
                print("Login successfully...")
    
                smtpObj.sendmail(sender, receivers, message)   # send mail to receiver      
                print ("Successfully sent email")

            except SMTPException: 
                print ("Error: unable to send email")
                print()
                
######### Send also whatsapp message to my friend  
### For that we import module pywhatkit , install it with "pip install pywhatkit"
            
            break
         
        else:
            cv2.putText(image, "I dont know, how r u?", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            
            
            print(" Your are not Owner, so w'll start Launch an  ec2 instance......!!! ")
            
# Launch an ec2 instances to my aws account       
            ec2 = boto3.resource('ec2')  #Establish connection to ec2 instance   
            instance = ec2.create_instances(
                    ImageId = 'ami-0ad704c126371a549',  # use the AMI-linux type image
                    MinCount = 1,
                    MaxCount = 1,
                    InstanceType = 't2.micro',
                    KeyName = 'key-for-rdp-os',  # try use the key pair available on aws acc
                    SubnetId = 'subnet-2762834c')
    
            instance_id = instance[0].id
            print ("The instance id is: ", instance_id)  #instance[0].id)
            print()
            time.sleep(5)

            conn = boto3.ec2.connect_to_region("ap-south-1")

###To stop instance with boto    
#stop = conn.stop_instances(instance_ids= instance_id) #['i-0638b07266fd59322'])
#print("The stopped instance is: ", stop)
    
            ## create volume of 5Gb     
            vol = conn.create_volume(5, "ap-south-1a") #launch EBS volume of 5GB
            print("Launched Volume is: ", vol)
            print()

##get The status of the volume available or not
            curr_vol = conn.get_all_volumes([vol.id])[0]   # get all volumes availables
            s = curr_vol.status          # get the status of volume launched
            print("Volume Status: ", s)
            print()

#Attach the volume created 
#if curr_vol is  None:
            time.sleep(100)
            conn.attach_volume (vol.id, instance_id, "/dev/sdx")  #Attach a volume to my instance created above
            print("Volume successfully attached...")
            print("Task Completed !!!!")
            break    

##Detach volume
#            det_vol = conn.detach_volume(vol.id, dry_run=False)
#            print(det_vol) #, "\n")
#            time.sleep(30)

##Delete Volume 
#            del_vol = conn.delete_volume( vol.id, dry_run=False )
#            print("Deleted Volume: ", del_vol)
#            print()

##To terminate instance 
#            t = conn.terminate_instances(instance_ids= instance_id)
#            print("Terminated intance: ", t)

            
    except:
        
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

 Your are not Owner, so w'll start Launch an  ec2 instance......!!! 
The instance id is:  i-03a11af0f0a654362

 Your are not Owner, so w'll start Launch an  ec2 instance......!!! 
The instance id is:  i-096438807a5ed58a0

Login successfully...
Successfully sent email


In [ ]:
"""
os.system("aws ec2 run-instances \
	--image-id  ami-0ad704c126371a549 \
	--count 1 \
	--instance-type t2.micro \
	--key-name key-sec-os   \
	--security-group-ids sg-02e74af0e4f2b66e1 \
	--subnet-id  subnet-2762834c   \
	--tag-specifications 'ResourceType=instance, Tags=[{key=Name,Value=Cli-OS}]'")



subprocess.getoutput('aws ec2 run-instances \
          --image-id  ami-0ad704c126371a549 \
          --count 1 \
          --instance-type t2.micro  \
          --key-name key-for-rdp-os  \
          --security-group-ids sg-02e74af0e4f2b66e1 \
          --subnet-id  subnet-2762834c \
          --tag-specifications 'ResourceType=instance, Tags=[{key=Name,Value=Cli-OS}]'

          ')

"""